In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import mlflow
import pandas as pd
import plotly.express as px

In [ ]:
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg = hydra.compose('train')

In [ ]:
cfg.logging

In [ ]:
client = mlflow.tracking.client.MlflowClient(tracking_uri=cfg.logging.mlflow.tracking_uri)

In [ ]:
def runs_to_df(runs):
    rows = []
    for r in runs:
        rmse_history = client.get_metric_history(r.info.run_id, 'Val/RMSE')

        min_rmse = min(rmse_history, key=lambda x: x.value) if len(rmse_history) > 0 else None


        rows.append({
            'model': r.data.params.get('model._target_', r.data.params.get('model/_target_', '')).split('.')[-1],
            'train_begin': r.data.params.get('split.train_begin', r.data.params.get('split/train_begin', '')),
            'val_begin': r.data.params.get('split.val_begin', r.data.params.get('split/val_begin', '')),
            'min_rmse': min_rmse.value if min_rmse else None,
            'start_time': r.info.start_time,
            'station_subset': r.data.params.get('dataset/station_set_file', r.data.params.get('dataset.station_set_file', '')),
            'run_name': r.data.tags['mlflow.runName'],
            'freeze_upper': 'True' == r.data.params.get('freeze_upper', False),
            'split_name': r.data.params.get('split.name', ''),
        })
        
    df = pd.DataFrame(rows)
    df['start_time'] = pd.to_datetime(df['start_time'], unit='ms')
        
    return df

In [ ]:
raw_model_runs = client.search_runs(
    experiment_ids="2",
    filter_string="tags.`mlflow.runName`='raw_gdps_backtest_bootstrap'",
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
)

In [ ]:
attention_runs = client.search_runs(
    experiment_ids="2",
    filter_string="tags.`mlflow.runName` LIKE 'attention_finetune_backtest%'",
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
)

In [ ]:
mos_runs = client.search_runs(
    experiment_ids="2",
    filter_string="tags.`mlflow.runName` LIKE 'emos_backtest_bootstrap_%D'",
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
)

In [ ]:
finetuning_runs = client.search_runs(
    experiment_ids="10",
    filter_string="tags.`mlflow.runName` LIKE 'finetune_20220527_02'",
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
)

In [ ]:
dfs = [
    runs_to_df(attention_runs),
    runs_to_df(mos_runs),
    runs_to_df(raw_model_runs),
]

In [ ]:
runs_to_df(mos_runs)

In [ ]:
dfs[2]

In [ ]:
df = pd.concat(dfs)
df['start_time'] = pd.to_datetime(df['start_time'])
df['train_begin'] = pd.to_datetime(df['train_begin'])
df['val_begin'] = pd.to_datetime(df['val_begin'])
df['train_length'] = df['val_begin'] - df['train_begin']

#df = df[df['val_begin'] >= '2020-08-01']

In [ ]:
fig = px.scatter(data_frame=df, x='train_length', y='min_rmse', color='run_name', labels={'min_rmse': 'RMSE on validation set (°C)'}, title='Validation loss for post-processing models on GDPS data')
#fig.add_hline(y=raw_model_metric, annotation_text='Raw GDPS')

In [ ]:
fig

In [ ]:
aggregated = df.groupby('run_name').agg({'min_rmse': 'mean', 'station_subset': 'count', 'train_length': 'first', 'model': 'first'})

In [ ]:
aggregated.sort_values(by=['model', 'train_length'])

In [ ]:
px.scatter(data_frame=aggregated, x='train_length', y='min_rmse', color='model')

In [ ]:
df_30d

In [ ]:
df_30d = df[df['run_name'].isin(['attention_finetune_backtest_30D', 'emos_backtest_bootstrap_30D'])]

In [ ]:
df_30d

In [ ]:
px.scatter(data_frame=df_30d[~df_30d['freeze_upper']], x='val_begin', y='min_rmse', hover_data=['freeze_upper', 'split_name', 'station_subset'], color='model', opacity=0.8)

In [ ]:
df_30d.groupby(['val_begin', 'freeze_upper']).agg({'min_rmse': 'mean', 'model': 'count'})

In [ ]:
df_30d.groupby(['model', 'freeze_upper', 'station_subset']).mean()

In [ ]:
df_30d['freeze_upper'] = df_30d['freeze_upper'].fillna(False)

In [ ]:
df_30d.groupby(['model', 'freeze_upper', 'station_subset']).count()

# Finetuning runs

In [ ]:
df = runs_to_df(finetuning_runs)

In [ ]:
df